# Final Project - Group 18
#####  Zili Bu - G36963059
#####  Mengxin Tan - G38541671
#####  Junyi Qian - G37449683
#####  Michael Lombardo - G47242943

## Brief Introduction

*We found the dataset from the Office of Foreign Labor Certification which is a division of the Department of Labor. 
The office is responsible for enforcing certain labor conditions and laws before employers can hire foreign workers. 
The most common mission is to verify whether foreign workers are qualified to obtain H1B visa.
We are aiming to analyze how factors such as wage, employer, case relative or position influence the result of a H1B application and we believe that analyzing the contents of the labor condition application would be helpful to both foreign students who are looking for jobs in United States and companies who are looking to attain wage data on their competitors.*

* Data Link: https://www.kaggle.com/trivedicharmi/h1b-disclosure-dataset#1.%20Master%20H1B%20Dataset.csv

## Business questions we are interested

* 1.How is status of a case affected by other data elements and can they identify a pattern in certification or denial?

* 2.Which companies request the most foreign (H1B) employees and how have the requests changed by year for these companies? Which industries do these companies work in?

* 3.Which industries offer the highest annual wages and how do the wages compare for competitors in the top occupations?  How do wages vary over location?

## Concerns about our dataset

* 1.We do not know the reason for how each case is decided on.

* 2.The data is exclusively of decisions made in 2016 and 2017.

* 3.The data provider takes out 13 columns due to privacy reasons. So we do not have full infomation about each row. 

### Load SQL 

In [1]:
!dropdb -U student week12

In [2]:
!createdb -U student week12

In [3]:
%load_ext sql

In [4]:
%sql postgresql://student@/week12

'Connected: student@week12'

### Wrangling data

```
filter type: custom rowType: single row: ISMISSING([EMPLOYER_STATE]) action: Delete
settype col: PREVAILING_WAGE type: Float
filter type: custom rowType: single row: ISMISSING([PW_UNIT_OF_PAY]) action: Delete
filter type: custom rowType: single row: ISMISMATCHED(PW_SOURCE_YEAR, ['Datetime','yy','yyyy']) action: Delete
settype col: WAGE_RATE_OF_PAY_FROM type: Float
settype col: WAGE_RATE_OF_PAY_TO type: Float
settype col: CASE_SUBMITTED_MONTH customType: 'Datetime','mm-dd-yy','mm*dd*yyyy' type: custom
settype col: CASE_SUBMITTED_MONTH type: Integer
derive type: single value: DATEFORMAT(DATE(CASE_SUBMITTED_YEAR, CASE_SUBMITTED_MONTH, CASE_SUBMITTED_DAY), 'yyyy-MM-dd') as: 'CASE_SUBMITTED_DATE'
settype col: CASE_SUBMITTED_YEAR type: Integer
drop col: CASE_SUBMITTED_DATE action: Drop
derive type: single value: DATEFORMAT(DATE(CASE_SUBMITTED_YEAR, CASE_SUBMITTED_MONTH, CASE_SUBMITTED_DAY), 'yyyy-MM-dd') as: 'CASE_SUBMITTED_DATE'
settype col: DECISION_YEAR type: Integer
derive type: single value: DATEFORMAT(DATE(DECISION_YEAR, DECISION_MONTH, DECISION_DAY), 'yyyy-MM-dd') as: 'DECISION_DATE'
derive type: single value: DATEDIF(CASE_SUBMITTED_DATE, DECISION_DATE, day) as: 'CASE_DECISION_DURATION'
settype col: WORKSITE_POSTAL_CODE type: String
settype col: PW_SOURCE_YEAR type: Integer
settype col: NAICS_CODE type: Integer
settype col: WORKSITE_POSTAL_CODE type: Integer
filter type: custom rowType: single row: ISMISSING([{H-1B_DEPENDENT}]) action: Delete
filter type: missing missing: WILLFUL_VIOLATOR action: Delete
filter type: missing missing: FULL_TIME_POSITION action: Delete
filter type: missing missing: WAGE_UNIT_OF_PAY action: Delete
filter type: missing missing: PW_SOURCE action: Delete
filter type: missing missing: PW_SOURCE_YEAR action: Delete
filter type: missing missing: EMPLOYER_NAME action: Delete
filter type: missing missing: PW_SOURCE_OTHER action: Delete
```

```
We used Excel to calculate the annual wage becaues the unit of payment is not consistent, 
with different units of year, week, month or biweekly.
We generate two date columns and add two more derived attributes which are duration and wage difference
```

### Load data from where it was uploaded (S3)

In [5]:
!wget https://aws-athena-query-results-644972362552-us-east-1.s3.amazonaws.com/h1b.csv

--2019-11-30 15:26:48--  https://aws-athena-query-results-644972362552-us-east-1.s3.amazonaws.com/h1b.csv
Resolving aws-athena-query-results-644972362552-us-east-1.s3.amazonaws.com (aws-athena-query-results-644972362552-us-east-1.s3.amazonaws.com)... 52.216.185.91
Connecting to aws-athena-query-results-644972362552-us-east-1.s3.amazonaws.com (aws-athena-query-results-644972362552-us-east-1.s3.amazonaws.com)|52.216.185.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106821965 (102M) [text/csv]
Saving to: ‘h1b.csv’

h1b.csv             100%[===================>] 101.87M  52.2MB/s    in 2.0s    

2019-11-30 15:26:50 (52.2 MB/s) - ‘h1b.csv’ saved [106821965/106821965]



### Have a glance of the dataset

In [6]:
!wc -l h1b.csv

517783 h1b.csv


In [7]:
!csvcut -n h1b.csv

  1: CASE_SUBMITTED_DAY
  2: CASE_SUBMITTED_MONTH
  3: CASE_SUBMITTED_YEAR
  4: CASE_SUBMITTED_DATE
  5: DECISION_DAY
  6: DECISION_MONTH
  7: DECISION_YEAR
  8: DECISION_DATE
  9: CASE_DECISION_DURATION
 10: VISA_CLASS
 11: EMPLOYER_NAME
 12: EMPLOYER_STATE
 13: EMPLOYER_COUNTRY
 14: SOC_NAME
 15: NAICS_CODE
 16: TOTAL_WORKERS
 17: FULL_TIME_POSITION
 18: PREVAILING_WAGE_ANNUAL
 19: PW_SOURCE
 20: PW_SOURCE_YEAR
 21: PW_SOURCE_OTHER
 22: WAGE_RATE_OF_PAY_FROM
 23: WAGE_RATE_OF_PAY_TO
 24: WAGE_ANNUAL
 25: WAGE_DIF
 26: H-1B_DEPENDENT
 27: WILLFUL_VIOLATOR
 28: WORKSITE_STATE
 29: WORKSITE_POSTAL_CODE
 30: CASE_STATUS


### Further explore data to understand data types of each column, length of values

In [9]:
# Take 1% of the whole dataset as a sample to see the basic stats
!head -n 5000 h1b.csv | csvstat 

  1. "CASE_SUBMITTED_DAY"

	Type of data:          Number
	Contains null values:  False
	Unique values:         31
	Smallest value:        1
	Largest value:         31
	Sum:                   128,859
	Mean:                  25.777
	Median:                28
	StDev:                 6.585
	Most common values:    27 (1247x)
	                       28 (1220x)
	                       29 (982x)
	                       30 (732x)
	                       3 (65x)

  2. "CASE_SUBMITTED_MONTH"

	Type of data:          Number
	Contains null values:  False
	Unique values:         12
	Smallest value:        1
	Largest value:         12
	Sum:                   41,904
	Mean:                  8.382
	Median:                9
	StDev:                 1.897
	Most common values:    9 (4207x)
	                       3 (332x)
	                       10 (118x)
	                       2 (85x)
	                       8 (54x)

  3. "CASE_SUBMITTED_YEAR"

	Type of data:          Number
	Contains null values:  False

In [10]:
# Take one important column form the whole dataset to see its basic stats
!xsv select 9 h1b.csv | xsv stats | xsv table 

field                   type     sum       min  max   min_length  max_length  mean               stddev
CASE_DECISION_DURATION  Integer  16236982  0    2214  1           4           31.35872239668375  127.91022142794715


### This is our Star-Schema of our dimentional model

In [9]:
from IPython.display import Image

In [101]:
Image(url = "https://data-project-picture.s3.amazonaws.com/Inkedfact_dimension_table.jpg")

### Create table for our data

In [5]:
%%sql
DROP TABLE IF EXISTS H1B;

CREATE TABLE H1B (
    CASE_SUBMITTED_DAY     INTEGER NOT NULL,
    CASE_SUBMITTED_MONTH   INTEGER NOT NULL,
    CASE_SUBMITTED_YEAR    INTEGER NOT NULL,
    CASE_SUBMITTED_DATE    DATE NOT NULL,
    DECISION_DAY           INTEGER NOT NULL,
    DECISION_MONTH         INTEGER NOT NULL,
    DECISION_YEAR          INTEGER NOT NULL,
    DECISION_DATE          DATE NOT NULL,
    CASE_DECISION_DURATION INTEGER NOT NULL,
    VISA_CLASS             VARCHAR(80) NOT NULL,
    EMPLOYER_NAME          VARCHAR(76) NOT NULL,
    EMPLOYER_STATE         VARCHAR(2) NOT NULL,
    EMPLOYER_COUNTRY       VARCHAR(24) NOT NULL,
    SOC_NAME               VARCHAR(33) NOT NULL,
    NAICS_CODE             INTEGER,
    TOTAL_WORKERS          INTEGER NOT NULL,
    FULL_TIME_POSITION     VARCHAR(1),
    PREVAILING_WAGE_ANNUAL NUMERIC,
    PW_SOURCE              VARCHAR(5),
    PW_SOURCE_YEAR         INTEGER,
    PW_SOURCE_OTHER        VARCHAR(150) NOT NULL,
    WAGE_RATE_OF_PAY_FROM  NUMERIC(10,2),
    WAGE_RATE_OF_PAY_TO    NUMERIC(10,2),
    WAGE_ANNUAL            NUMERIC(20,2),
    WAGE_DIF               NUMERIC,
    H1B_DEPENDENT          VARCHAR(1),
    WILLFUL_VIOLATOR       VARCHAR(1),
    WORKSITE_STATE         VARCHAR(2) NOT NULL,
    WORKSITE_POSTAL_CODE   VARCHAR,
    CASE_STATUS            VARCHAR(18) CHECK (CASE_STATUS in ('CERTIFIED','CERTIFIEDWITHDRAWN','WITHDRAWN', 'DENIED'))
);

 * postgresql://student@/week12
Done.
Done.


[]

### Set data into a temporary file so that it is reproducible by others running this code and load into our table

In [6]:
!cp h1b.csv /tmp/h1b.csv

In [7]:
%%sql
COPY H1B FROM '/tmp/h1b.csv'
CSV
HEADER;

 * postgresql://student@/week12
517782 rows affected.


[]

In [19]:
%%sql
SELECT * FROM H1B
LIMIT 5;

 * postgresql://student@/week12
5 rows affected.


case_status,case_key,position_key,wage_key,employer_key
CERTIFIED,13558,107109,226850,50148
CERTIFIED,14441,104153,111462,64404
WITHDRAWN,8118,18079,92894,14838
CERTIFIED,16093,96795,117156,52458
WITHDRAWN,2479,21986,82362,60875


## Now separating the data into dimensions. For each table we: Select data, create table, insert data to table, and update fact table.

### 1. Case Dimension 

In [8]:
%%sql
select DISTINCT case_submitted_day, case_submitted_month, case_submitted_year, case_submitted_date, decision_day, decision_month, decision_year, decision_date, case_decision_duration, visa_class 
from H1B
limit 10;

 * postgresql://student@/week12
10 rows affected.


case_submitted_day,case_submitted_month,case_submitted_year,case_submitted_date,decision_day,decision_month,decision_year,decision_date,case_decision_duration,visa_class
9,3,2016,2016-03-09,6,12,2016,2016-12-06,272,H1B
7,3,2016,2016-03-07,29,4,2017,2017-04-29,418,H1B
1,3,2016,2016-03-01,5,4,2017,2017-04-05,400,H1B
15,4,2015,2015-04-15,9,1,2017,2017-01-09,635,H1B
27,4,2015,2015-04-27,18,1,2017,2017-01-18,632,H1B
3,3,2016,2016-03-03,18,1,2017,2017-01-18,321,H1B
14,5,2015,2015-05-14,7,11,2016,2016-11-07,543,H1B
4,10,2016,2016-10-04,20,3,2017,2017-03-20,167,H1B
6,12,2016,2016-12-06,20,1,2017,2017-01-20,45,H1B
30,6,2016,2016-06-30,30,12,2016,2016-12-30,183,H1B


In [9]:
%%sql
DROP TABLE IF EXISTS CASES;

CREATE TABLE CASES (
    KEY                    SERIAL PRIMARY KEY,
    CASE_SUBMITTED_DAY     INTEGER NOT NULL,
    CASE_SUBMITTED_MONTH   INTEGER NOT NULL,
    CASE_SUBMITTED_YEAR    INTEGER NOT NULL,
    CASE_SUBMITTED_DATE    DATE NOT NULL,
    DECISION_DAY           INTEGER NOT NULL,
    DECISION_MONTH         INTEGER NOT NULL,
    DECISION_YEAR          INTEGER NOT NULL,
    DECISION_DATE          DATE NOT NULL,
    CASE_DECISION_DURATION INTEGER NOT NULL,
    VISA_CLASS             VARCHAR(80) NOT NULL
);

 * postgresql://student@/week12
Done.
Done.


[]

In [10]:
%%sql
INSERT INTO CASES (case_submitted_day, case_submitted_month, case_submitted_year, case_submitted_date, decision_day, decision_month, decision_year, decision_date, case_decision_duration, visa_class)
SELECT DISTINCT case_submitted_day, case_submitted_month, case_submitted_year, case_submitted_date, decision_day, decision_month, decision_year, decision_date, case_decision_duration, visa_class  
FROM H1B;

 * postgresql://student@/week12
23961 rows affected.


[]

In [11]:
%%sql
SELECT * FROM CASES
LIMIT 10;

 * postgresql://student@/week12
10 rows affected.


key,case_submitted_day,case_submitted_month,case_submitted_year,case_submitted_date,decision_day,decision_month,decision_year,decision_date,case_decision_duration,visa_class
1,9,3,2016,2016-03-09,6,12,2016,2016-12-06,272,H1B
2,7,3,2016,2016-03-07,29,4,2017,2017-04-29,418,H1B
3,1,3,2016,2016-03-01,5,4,2017,2017-04-05,400,H1B
4,15,4,2015,2015-04-15,9,1,2017,2017-01-09,635,H1B
5,27,4,2015,2015-04-27,18,1,2017,2017-01-18,632,H1B
6,3,3,2016,2016-03-03,18,1,2017,2017-01-18,321,H1B
7,14,5,2015,2015-05-14,7,11,2016,2016-11-07,543,H1B
8,4,10,2016,2016-10-04,20,3,2017,2017-03-20,167,H1B
9,6,12,2016,2016-12-06,20,1,2017,2017-01-20,45,H1B
10,30,6,2016,2016-06-30,30,12,2016,2016-12-30,183,H1B


In [12]:
%%sql
ALTER TABLE H1B
ADD COLUMN CASE_KEY INTEGER,
ADD CONSTRAINT FK_CASE_KEY
    FOREIGN KEY (CASE_KEY)
    REFERENCES CASES(KEY);

 * postgresql://student@/week12
Done.


[]

In [13]:
%%sql
UPDATE H1B
SET CASE_KEY = CASES.KEY
FROM CASES
WHERE H1B.case_submitted_day = CASES.case_submitted_day AND H1B.case_submitted_month = CASES.case_submitted_month 
AND H1B.case_submitted_year = CASES.case_submitted_year AND H1B.case_submitted_date = CASES.case_submitted_date
AND H1B.decision_day = CASES.decision_day AND H1B.decision_month = CASES.decision_month
AND H1B.decision_year = CASES.decision_year AND H1B.decision_date = CASES.decision_date
AND H1B.case_decision_duration = CASES.case_decision_duration AND H1B.visa_class = CASES.visa_class;

 * postgresql://student@/week12
517782 rows affected.


[]

### 2. Position Dimension

In [14]:
%%sql
SELECT DISTINCT SOC_NAME, NAICS_CODE, FULL_TIME_POSITION, WORKSITE_STATE, WORKSITE_POSTAL_CODE
FROM H1B
ORDER BY SOC_NAME
LIMIT 10;

 * postgresql://student@/week12
10 rows affected.


soc_name,naics_code,full_time_position,worksite_state,worksite_postal_code
ACCOUNTANTS,541511,Y,OH,45209
ACCOUNTANTS,541511,Y,IL,60606
ACCOUNTANTS,511210,Y,CA,94105
ACCOUNTANTS,541511,Y,MN,55343
ACCOUNTANTS,541511,Y,IA,50392
ACCOUNTANTS,325211,Y,MA,1532
ACCOUNTANTS,541613,Y,NY,11209
ACCOUNTANTS,541612,Y,GA,30022
ACCOUNTANTS,621111,Y,IN,46123
ACCOUNTANTS,522320,Y,GA,30004


In [15]:
%%sql
DROP TABLE IF EXISTS POSITION CASCADE;

CREATE TABLE POSITION (
    KEY                    SERIAL PRIMARY KEY,
    SOC_NAME               VARCHAR(33) NOT NULL,
    NAICS_CODE             INTEGER,
    FULL_TIME_POSITION     VARCHAR(1),
    WORKSITE_STATE         VARCHAR(2) NOT NULL,
    WORKSITE_POSTAL_CODE   VARCHAR
);

 * postgresql://student@/week12
Done.
Done.


[]

In [16]:
%%sql
INSERT INTO POSITION (SOC_NAME, NAICS_CODE, FULL_TIME_POSITION, WORKSITE_STATE, WORKSITE_POSTAL_CODE)
SELECT DISTINCT SOC_NAME, NAICS_CODE, FULL_TIME_POSITION, WORKSITE_STATE, WORKSITE_POSTAL_CODE
FROM H1B;

 * postgresql://student@/week12
137581 rows affected.


[]

In [17]:
%%sql
ALTER TABLE H1B
ADD COLUMN POSITION_KEY INTEGER,
ADD CONSTRAINT FK_POSITION_KEY
    FOREIGN KEY (POSITION_KEY)
    REFERENCES POSITION (KEY);

 * postgresql://student@/week12
Done.


[]

In [26]:
%%sql 
SELECT * FROM H1B
LIMIT 3;

 * postgresql://student@/week12
3 rows affected.


case_submitted_day,case_submitted_month,case_submitted_year,case_submitted_date,decision_day,decision_month,decision_year,decision_date,case_decision_duration,visa_class,employer_name,employer_state,employer_country,soc_name,naics_code,total_workers,full_time_position,prevailing_wage_annual,pw_source,pw_source_year,pw_source_other,wage_rate_of_pay_from,wage_rate_of_pay_to,wage_annual,wage_dif,h1b_dependent,willful_violator,worksite_state,worksite_postal_code,case_status,case_key,position_key
27,9,2016,2016-09-27,3,10,2016,2016-10-03,6,H1B,STAPLES INC,MA,UNITED STATES OF AMERICA,EDUCATION,453210,1,Y,120224,OES,2016,OFLC ONLINE DATA CENTER,125000.00,0.00,125000.00,4776,N,N,MA,1702,CERTIFIED,16700,None
27,9,2016,2016-09-27,3,10,2016,2016-10-03,6,H1B,WIPRO LIMITED,NJ,UNITED STATES OF AMERICA,MANAGERS,541511,1,Y,60000,OES,2016,OFLC ONLINE DATA CENTER,60000.00,0.00,60000.00,0,Y,N,MO,63043,CERTIFIED,16700,None
3,10,2016,2016-10-03,5,10,2016,2016-10-05,2,H1B,GLOBAL BMU LLC,NY,UNITED STATES OF AMERICA,MARKETING,2381,1,Y,150009.6,SCA,2014,NONE,64.31,0.00,133764.80,-16244.8,N,N,NY,10011,DENIED,3258,None


In [18]:
%%sql
UPDATE H1B
SET POSITION_KEY = POSITION.KEY
FROM POSITION
WHERE H1B.SOC_NAME = POSITION.SOC_NAME AND H1B.NAICS_CODE = POSITION.NAICS_CODE
AND H1B.full_time_position = POSITION.FULL_TIME_POSITION
AND H1B.worksite_state = POSITION.WORKSITE_STATE AND H1B.worksite_postal_code = POSITION.WORKSITE_POSTAL_CODE;

 * postgresql://student@/week12
517780 rows affected.


[]

### 3. Wage Dimension

In [19]:
%%sql
SELECT DISTINCT PREVAILING_WAGE_ANNUAL, PW_SOURCE, PW_SOURCE_YEAR, PW_SOURCE_OTHER, WAGE_RATE_OF_PAY_FROM, WAGE_RATE_OF_PAY_TO, WAGE_ANNUAL, WAGE_DIF  
FROM H1B
ORDER BY WAGE_DIF
LIMIT 5;

 * postgresql://student@/week12
5 rows affected.


prevailing_wage_annual,pw_source,pw_source_year,pw_source_other,wage_rate_of_pay_from,wage_rate_of_pay_to,wage_annual,wage_dif
297785280,OES,2016,OFLC ONLINE DATA CENTER,155000.00,165000.00,155000.00,-297630280
205892960,OES,2016,OFLC ONLINE DATA CENTER,98987.00,111000.00,98987.00,-205793973
195813280,OES,2016,OFLC ONLINE DATA CENTER,106000.00,0.00,106000.00,-195707280
181319840,OES,2016,OFLC ONLINE DATA CENTER,96187.00,0.00,96187.00,-181223653
172839680,OES,2016,OFLC ONLINE DATA CENTER,85000.00,0.00,85000.00,-172754680


In [20]:
%%sql
DROP TABLE IF EXISTS WAGE;

CREATE TABLE WAGE (
    KEY                          SERIAL PRIMARY KEY,
    PREVAILING_WAGE_ANNUAL       NUMERIC,
    PW_SOURCE                    VARCHAR(5),
    PW_SOURCE_YEAR               INTEGER,
    PW_SOURCE_OTHER              VARCHAR(150) NOT NULL,
    WAGE_RATE_OF_PAY_FROM        NUMERIC(10,2),
    WAGE_RATE_OF_PAY_TO          NUMERIC(10,2),
    WAGE_ANNUAL                  NUMERIC(20,2),
    WAGE_DIF                     NUMERIC
);

 * postgresql://student@/week12
Done.
Done.


[]

In [21]:
%%sql
INSERT INTO WAGE (PREVAILING_WAGE_ANNUAL, PW_SOURCE, PW_SOURCE_YEAR, PW_SOURCE_OTHER, WAGE_RATE_OF_PAY_FROM, WAGE_RATE_OF_PAY_TO, WAGE_ANNUAL, WAGE_DIF)
SELECT DISTINCT PREVAILING_WAGE_ANNUAL, PW_SOURCE, PW_SOURCE_YEAR, PW_SOURCE_OTHER, WAGE_RATE_OF_PAY_FROM, WAGE_RATE_OF_PAY_TO, WAGE_ANNUAL, WAGE_DIF
FROM H1B;

 * postgresql://student@/week12
232908 rows affected.


[]

In [32]:
%%sql
SELECT * FROM WAGE
ORDER BY WAGE_DIF
LIMIT 3;

 * postgresql://student@/week12
3 rows affected.


key,prevailing_wage_annual,pw_source,pw_source_year,pw_source_other,wage_rate_of_pay_from,wage_rate_of_pay_to,wage_annual,wage_dif
232908,297785280,OES,2016,OFLC ONLINE DATA CENTER,155000.00,165000.00,155000.00,-297630280
232907,205892960,OES,2016,OFLC ONLINE DATA CENTER,98987.00,111000.00,98987.00,-205793973
232906,195813280,OES,2016,OFLC ONLINE DATA CENTER,106000.00,0.00,106000.00,-195707280


In [22]:
%%sql
ALTER TABLE H1B
ADD COLUMN WAGE_KEY INTEGER,
ADD CONSTRAINT FK_WAGE_KEY
    FOREIGN KEY (WAGE_KEY)
    REFERENCES WAGE(KEY);

 * postgresql://student@/week12
Done.


[]

In [23]:
%%sql
UPDATE H1B
SET WAGE_KEY = WAGE.KEY
FROM WAGE
WHERE H1B.prevailing_wage_annual = WAGE.prevailing_wage_annual AND H1B.pw_source = WAGE.pw_source AND
H1B.PW_SOURCE_YEAR = WAGE.PW_SOURCE_YEAR AND H1B.PW_SOURCE_OTHER = WAGE.PW_SOURCE_OTHER AND
H1B.WAGE_RATE_OF_PAY_FROM = WAGE.WAGE_RATE_OF_PAY_FROM AND H1B.WAGE_RATE_OF_PAY_TO = WAGE.WAGE_RATE_OF_PAY_TO
AND H1B.wage_annual = WAGE.wage_annual AND H1B.wage_dif = WAGE.wage_dif
;

 * postgresql://student@/week12
517782 rows affected.


[]

### 4. Employer dimension

In [24]:
%%sql
DROP TABLE IF EXISTS EMPLOYER;

CREATE TABLE EMPLOYER(
    KEY SERIAL PRIMARY KEY,
    EMPLOYER_STATE VARCHAR(2), 
    EMPLOYER_NAME VARCHAR, 
    EMPLOYER_COUNTRY VARCHAR, 
    TOTAL_WORKERS INTEGER,
    H1B_DEPENDENT VARCHAR(1), 
    WILLFUL_VIOLATOR VARCHAR(1)
);

 * postgresql://student@/week12
Done.
Done.


[]

In [25]:
%%sql
INSERT INTO EMPLOYER (EMPLOYER_STATE, EMPLOYER_NAME, EMPLOYER_COUNTRY, TOTAL_WORKERS, H1B_DEPENDENT, WILLFUL_VIOLATOR)
SELECT DISTINCT EMPLOYER_STATE, EMPLOYER_NAME, EMPLOYER_COUNTRY, TOTAL_WORKERS, H1B_DEPENDENT, WILLFUL_VIOLATOR
FROM H1B;

 * postgresql://student@/week12
64563 rows affected.


[]

In [37]:
%%sql
SELECT * FROM EMPLOYER
LIMIT 3;

 * postgresql://student@/week12
3 rows affected.


key,employer_state,employer_name,employer_country,total_workers,h1b_dependent,willful_violator
1,AK,ABS BILLING SERVICES INC,UNITED STATES OF AMERICA,1,N,N
2,AK,ALASKA NATIVE TRIBAL HEALTH CONSORTIUM,UNITED STATES OF AMERICA,1,N,N
3,AK,ALTMAN ROGERS and CO,UNITED STATES OF AMERICA,1,N,N


In [26]:
%%sql
ALTER TABLE H1B
ADD COLUMN EMPLOYER_KEY INTEGER,
ADD CONSTRAINT FK_EMPLOYER_KEY
    FOREIGN KEY (EMPLOYER_KEY)
    REFERENCES EMPLOYER (KEY);

 * postgresql://student@/week12
Done.


[]

In [27]:
%%sql
UPDATE H1B
SET EMPLOYER_KEY = EMPLOYER.KEY
FROM EMPLOYER
WHERE H1B.employer_state = EMPLOYER.employer_state AND H1B.employer_name =  EMPLOYER.employer_name AND
H1B.employer_country =  EMPLOYER.employer_country AND H1B.TOTAL_WORKERS = EMPLOYER.TOTAL_WORKERS AND H1B.h1b_dependent = EMPLOYER.h1b_dependent AND
H1B.willful_violator = EMPLOYER.willful_violator;

 * postgresql://student@/week12
517782 rows affected.


[]

### We need to drop these columns now from the initial H1B table and just use the keys to reference.  The H1B table is now our fact table.

In [28]:
%%sql
ALTER TABLE H1B
DROP COLUMN case_submitted_day, 
DROP COLUMN case_submitted_month,
DROP COLUMN case_submitted_year, 
DROP COLUMN case_submitted_date,
DROP COLUMN decision_day,
DROP COLUMN decision_month,
DROP COLUMN decision_year,
DROP COLUMN decision_date, 
DROP COLUMN case_decision_duration, 
DROP COLUMN visa_class, 
DROP COLUMN employer_name, 
DROP COLUMN employer_state, 
DROP COLUMN employer_country, 
DROP COLUMN soc_name, 
DROP COLUMN naics_code, 
DROP COLUMN total_workers, 
DROP COLUMN full_time_position, 
DROP COLUMN prevailing_wage_annual, 
DROP COLUMN pw_source, 
DROP COLUMN pw_source_year, 
DROP COLUMN pw_source_other, 
DROP COLUMN wage_rate_of_pay_from, 
DROP COLUMN wage_rate_of_pay_to, 
DROP COLUMN wage_annual,
DROP COLUMN wage_dif, 
DROP COLUMN h1b_dependent, 
DROP COLUMN willful_violator, 
DROP COLUMN worksite_state, 
DROP COLUMN worksite_postal_code;

 * postgresql://student@/week12
Done.


[]

In [42]:
%%sql
SELECT * FROM H1B
LIMIT 5;

 * postgresql://student@/week12
5 rows affected.


case_status,case_key,position_key,wage_key,employer_key
WITHDRAWN,22465,28144,107213,14838
WITHDRAWN,11895,112901,64688,63571
CERTIFIED,15827,80122,94243,60571
DENIED,10642,73796,23562,50074
CERTIFIED,11651,28057,89827,29625


## Business Questions

## Q1 - How is status of a case affected by other data elements and can they identify a pattern in certification or denial?

### 1.1 Finding the pattern about whether a case will be certified or denied in terms of Willful_Violator status

In [77]:
%%sql 
(SELECT employer.willful_violator, H1B.case_status, count(case_status)
FROM H1B
JOIN EMPLOYER
    ON H1B.EMPLOYER_KEY = EMPLOYER.KEY
WHERE willful_violator IN ('Y')
GROUP BY willful_violator, case_status)
UNION
(SELECT employer.willful_violator, H1B.case_status, count(case_status)
FROM H1B
JOIN EMPLOYER
    ON H1B.EMPLOYER_KEY = EMPLOYER.KEY
WHERE willful_violator IN ('N')
GROUP BY willful_violator, case_status)
ORDER BY 1 ASC, 2 ASC;

 * postgresql://student@/week12
8 rows affected.


willful_violator,case_status,count
N,CERTIFIED,459922
N,CERTIFIEDWITHDRAWN,35471
N,DENIED,6277
N,WITHDRAWN,15689
Y,CERTIFIED,277
Y,CERTIFIEDWITHDRAWN,112
Y,DENIED,15
Y,WITHDRAWN,19


In [38]:
from IPython.display import Image

In [102]:
Image(url = "https://data-project-picture.s3.amazonaws.com/Comp_WV_N.png")

In [103]:
Image(url = "https://data-project-picture.s3.amazonaws.com/Comp_WV_Y.png")

### We can see from the above charts with the following conclusions:
* When employers have not been identified as previous violators, 88.9% of cases are certified.  When employers have been identified as previous violators, only 65.5% of cases are certified.
* 6.8% were certified withdrawn vs 26.5% and 1.2% were denied vs 3.5%


### 1.2 Finding the pattern about whether a case will be certified or denied in terms of different months

Total number of cases in different status:

In [78]:
%%sql
SELECT h1b.case_status, count(case_status) AS number_of_cases
FROM H1B
GROUP BY CASE_STATUS;

 * postgresql://student@/week12
4 rows affected.


case_status,number_of_cases
WITHDRAWN,15708
DENIED,6292
CERTIFIEDWITHDRAWN,35583
CERTIFIED,460199


Certified cases in different months:

In [36]:
%%sql
SELECT H1B.case_status, CASES.CASE_SUBMITTED_MONTH, count(*)
FROM H1B 
JOIN CASES ON H1B.CASE_KEY = CASES.KEY
WHERE case_status IN ('CERTIFIED', 'CERTIFIEDWITHDRAWN')
GROUP BY case_status, CASE_SUBMITTED_MONTH
ORDER BY 1 ASC, 2 ASC;

 * postgresql://student@/week12
22 rows affected.


case_status,case_submitted_month,count
CERTIFIED,1,34956
CERTIFIED,2,73419
CERTIFIED,3,190447
CERTIFIED,4,29161
CERTIFIED,5,29517
CERTIFIED,6,24047
CERTIFIED,9,4496
CERTIFIED,10,24003
CERTIFIED,11,24441
CERTIFIED,12,25712


In [104]:
Image(url = "https://data-project-picture.s3.amazonaws.com/total_cases_per_month.png")

### There is a pattern of monthly application
* It is apparent that March has the most applicants of H1B, whether the case is certified or certifies-withdrawn. It is probably because deadline of applying H1B visa is in April. And most applicants choose to submit their application in February and March.

### 1.3 The difference between annual wage and minimum wage affects the case status

In [17]:
%%sql
SELECT CASE_STATUS, count(case_key)
FROM H1B
JOIN WAGE ON H1B.WAGE_KEY = WAGE.KEY
WHERE WAGE.WAGE_DIF < 0
GROUP BY CASE_STATUS;

 * postgresql://student@/week12
3 rows affected.


case_status,count
CERTIFIED,2
DENIED,1488
WITHDRAWN,309


In [39]:
Image(url = 'https://data-project-picture.s3.amazonaws.com/lower_wage_status.png')

* When the annual wage is lower than the minimum wage required by the government, the case is mostly denied or withdrawn.

## Q2 - Which companies request the most foreign (H1B) employees and how have the requests changed by year for these companies? Which industries do these companies work in?

### 2.1 Finding top 5 number of total workers requested by the companies

In [33]:
%%sql
SELECT EMPLOYER_NAME, sum(TOTAL_WORKERS) AS WORKERS_REQUESTED
FROM H1B
JOIN EMPLOYER ON H1B.EMPLOYER_KEY = EMPLOYER.KEY
JOIN CASES ON H1B.CASE_KEY = CASES.KEY
GROUP BY EMPLOYER_NAME
ORDER BY 2 DESC
LIMIT 5;

 * postgresql://student@/week12
5 rows affected.


employer_name,workers_requested
DELOITTE CONSULTING LLP,82266
COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION,46771
HCL AMERICA INC,28772
APPLE INC,26460
MPHASIS CORPORATION,19595


### Now take these employers and view by case_submitted_year

In [87]:
%%sql
SELECT EMPLOYER_NAME, case_submitted_year, sum(TOTAL_WORKERS) as workers
FROM H1B
JOIN EMPLOYER ON H1B.EMPLOYER_KEY = EMPLOYER.KEY
JOIN CASES ON H1B.CASE_KEY = CASES.KEY
WHERE EMPLOYER_NAME in ('DELOITTE CONSULTING LLP', 'COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION','HCL AMERICA INC', 'APPLE INC', 'MPHASIS CORPORATION')
GROUP BY case_submitted_year, EMPLOYER_NAME
ORDER BY 1 ASC, 2 ASC, 3 DESC;

 * postgresql://student@/week12
14 rows affected.


employer_name,case_submitted_year,workers
APPLE INC,2013,3
APPLE INC,2014,26
APPLE INC,2015,46
APPLE INC,2016,1526
APPLE INC,2017,24859
COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION,2015,280
COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION,2016,21880
COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION,2017,24611
DELOITTE CONSULTING LLP,2016,14440
DELOITTE CONSULTING LLP,2017,67826


In [105]:
Image(url = "https://data-project-picture.s3.amazonaws.com/workers_top5.png")

* With the time of each year, there is a increasing trend of the H1B workers required by these five companies. 
* From 2016 to 2017, there is a sharp boom of number of workers in APPLE and DELOITTE, 1500% and 369% respectively. While COGNIZANT only has a soft increase of 12.5%.

### 2.2 Finding the spread distribution for industries for the 5 top companies with most employees

In [18]:
%%sql
SELECT EMPLOYER_NAME, SOC_NAME, sum(TOTAL_WORKERS) as workers
FROM H1B
JOIN EMPLOYER ON H1B.EMPLOYER_KEY = EMPLOYER.KEY
JOIN POSITION ON H1B.POSITION_KEY = POSITION.KEY
WHERE EMPLOYER_NAME in ('DELOITTE CONSULTING LLP', 'COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION','HCL AMERICA INC', 'APPLE INC', 'MPHASIS CORPORATION')
GROUP BY SOC_NAME, EMPLOYER_NAME
HAVING sum(TOTAL_WORKERS) > 200
ORDER BY 1 ASC, 2 ASC, 3 DESC;

 * postgresql://student@/week12
35 rows affected.


employer_name,soc_name,workers
APPLE INC,ANALYSTS,25414
APPLE INC,BUSINESS OPERATIONS SPECIALIST,282
APPLE INC,COMPUTER OCCUPATION,278
COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION,ACCOUNTANTS,684
COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION,ANALYSTS,23521
COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION,ARCHITECTURE,612
COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION,BUSINESS OPERATIONS SPECIALIST,498
COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION,CHIEF EXECUTIVES,345
COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION,COACHES AND SCOUTS,377
COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION,COMPUTER OCCUPATION,16652


In [40]:
Image(url = 'https://data-project-picture.s3.amazonaws.com/top_5_occupation.png')

* The five companies have businesses in a very wide range. 
* It is apparent to see that computer occupation have way more numbers of employees in each company, followed by analysts and scientists.

## Q3 - Which industries offer the highest annual wages and how do the wages compare for competitors in the top occupations?  How do wages vary over location?

### 3.1 Finding the top 5 occupations with highest average annual wage

In [92]:
%%sql
SELECT SOC_NAME, round(avg(WAGE_ANNUAL),2) AS AVG_ANNUAL_WAGE
FROM H1B
JOIN POSITION ON H1B.POSITION_KEY = POSITION.KEY
JOIN WAGE ON H1B.WAGE_KEY = WAGE.KEY
WHERE CASE_STATUS = 'CERTIFIED'
GROUP BY SOC_NAME
ORDER BY 2 DESC
LIMIT 5;

 * postgresql://student@/week12
5 rows affected.


soc_name,avg_annual_wage
MANAGERS,118619.92
ANALYSTS,100603.98
DOCTORS,96327.29
ACCOUNTANTS,92599.80
FIRST LINE SUPERVISORS,90207.32


In [107]:
Image(url = "https://data-project-picture.s3.amazonaws.com/top5_occupation_wage.png")

### 3.2 Finding the most powerful competitors in three industries with most employees and comparing the average wage of these companies

In [6]:
%%sql
(SELECT SOC_NAME, EMPLOYER_NAME, sum(TOTAL_WORKERS) AS WORKERS_REQUESTED, ROUND(AVG(WAGE_ANNUAL)) AS AVG_SALARY
FROM H1B
JOIN EMPLOYER ON H1B.EMPLOYER_KEY = EMPLOYER.KEY
JOIN CASES ON H1B.CASE_KEY = CASES.KEY
JOIN POSITION ON H1B.POSITION_KEY = POSITION.KEY
JOIN WAGE ON H1B.WAGE_KEY = WAGE.KEY
WHERE SOC_NAME IN ('ANALYSTS') AND CASE_STATUS IN ('CERTIFIED', 'CERTIFIEDWITHDRAWN')
GROUP BY SOC_NAME, EMPLOYER_NAME
ORDER BY 1, 3 DESC
LIMIT 3)
UNION
(SELECT SOC_NAME, EMPLOYER_NAME, sum(TOTAL_WORKERS) AS WORKERS_REQUESTED, ROUND(AVG(WAGE_ANNUAL)) AS AVG_SALARY
FROM H1B
JOIN EMPLOYER ON H1B.EMPLOYER_KEY = EMPLOYER.KEY
JOIN CASES ON H1B.CASE_KEY = CASES.KEY
JOIN POSITION ON H1B.POSITION_KEY = POSITION.KEY
JOIN WAGE ON H1B.WAGE_KEY = WAGE.KEY
WHERE SOC_NAME IN ('DOCTORS') AND CASE_STATUS IN ('CERTIFIED', 'CERTIFIEDWITHDRAWN')
GROUP BY SOC_NAME, EMPLOYER_NAME
ORDER BY 1, 3 DESC
LIMIT 3)
UNION
(SELECT SOC_NAME, EMPLOYER_NAME, sum(TOTAL_WORKERS) AS WORKERS_REQUESTED, ROUND(AVG(WAGE_ANNUAL)) AS AVG_SALARY
FROM H1B
JOIN EMPLOYER ON H1B.EMPLOYER_KEY = EMPLOYER.KEY
JOIN CASES ON H1B.CASE_KEY = CASES.KEY
JOIN POSITION ON H1B.POSITION_KEY = POSITION.KEY
JOIN WAGE ON H1B.WAGE_KEY = WAGE.KEY
WHERE SOC_NAME IN ('MANAGERS') AND CASE_STATUS IN ('CERTIFIED', 'CERTIFIEDWITHDRAWN')
GROUP BY SOC_NAME, EMPLOYER_NAME
ORDER BY 1, 3 DESC
LIMIT 3)
ORDER BY SOC_NAME;

 * postgresql://student@/week12
9 rows affected.


soc_name,employer_name,workers_requested,avg_salary
ANALYSTS,APPLE INC,25259,132636
ANALYSTS,COGNIZANT TECHNOLOGY SOLUTIONS US CORPORATION,23521,75673
ANALYSTS,DELOITTE CONSULTING LLP,14445,90208
DOCTORS,SERVICENOW INC,630,106157
DOCTORS,DELOITTE CONSULTING LLP,990,79017
DOCTORS,QUALCOMM DATACENTER TECHNOLOGIES INC,700,108397
MANAGERS,WIPRO LLC,254,75426
MANAGERS,ZENSAR TECHNOLOGIES INC,281,90828
MANAGERS,WIPRO LIMITED,1074,73462


* We take the three postions with most workers and find out the top three companies within these industries. We and employers can utilize this analysis to get a picture of what other competitors offer in the same occupation.

In [10]:
Image(url = 'https://data-project-picture.s3.amazonaws.com/top_analyst.png')

* We can see Apple pays about 30% more than Deloitte and 43% more than Cognizant Tech in analysts position. We do not have details of analyst position, but we believe that analyst could describe many different positions which may explain the variation of the payment.

In [11]:
Image(url = 'https://data-project-picture.s3.amazonaws.com/top_doctors.png')

* For doctors we can see that the average wage is also quite high however Qualcomm and Servicenow offer quite a bit more than Deloitte.  The spread of total workers requested is relatively stable.

In [12]:
Image(url = 'https://data-project-picture.s3.amazonaws.com/top_manager.png')

* Zensar appears to offer the most for managers while Wipro Limited requests the most managers as employees.  It is difficult to make many conclusions as 'managers' could be within many different occupations where wages vary.

### 3.3 Within the 5 top states with most H1B employees, find the trend of average annual wage in recent years

* Find the top 5 states with most H1B employees

In [75]:
%%sql
SELECT EMPLOYER_STATE, SUM(TOTAL_WORKERS)
FROM H1B
JOIN EMPLOYER ON H1B.EMPLOYER_KEY = EMPLOYER.KEY
JOIN CASES ON H1B.CASE_KEY = CASES.KEY
GROUP BY 1
ORDER BY SUM DESC
LIMIT 5;

 * postgresql://student@/week12
5 rows affected.


employer_state,sum
CA,245038
PA,140534
TX,133619
NJ,91595
NY,66704


* Find average annual wage in these 5 states

In [11]:
%%sql
SELECT EMPLOYER_STATE, ROUND(AVG(WAGE_ANNUAL),2) AS AVG_ANNUAL_WAGE, CASE_SUBMITTED_YEAR
FROM H1B
JOIN EMPLOYER ON H1B.EMPLOYER_KEY = EMPLOYER.KEY
JOIN WAGE ON H1B.WAGE_KEY = WAGE.KEY
JOIN CASES ON H1B.CASE_KEY = CASES.KEY
WHERE EMPLOYER_STATE IN ('CA','PA','TX','NJ','NY') AND CASE_STATUS IN ('CERTIFIED','CERTIFIEDWITHDRAWN')
GROUP BY 1, 3
ORDER BY 1 ASC, 3 ASC;

 * postgresql://student@/week12
28 rows affected.


employer_state,avg_annual_wage,case_submitted_year
CA,71000.00,2011
CA,72800.00,2012
CA,91836.56,2013
CA,98184.84,2014
CA,104305.76,2015
CA,107613.12,2016
CA,103642.73,2017
NJ,68674.67,2012
NJ,64721.83,2013
NJ,70376.07,2014


In [45]:
Image(url = "https://data-project-picture.s3.amazonaws.com/top5_states_total_workers_fixed.png")

* We can see from the line graph that in these five states with most H1B employees, the average annual wage is  increasing with time in recent years.  Wages are highest in California and New York - which matches the high cost of living in these areas.
* We are only interested in the cases that have been certified or certified-withdraw, because we identified some input errors for wages which led to a withdraw or denial.
* In the graph above we can see that wages drop from 2016 to 2017.  Below we run a query of the prevailing wage which is the  government required minimum wage for different job types.  We can see from the query below that the government dropped the prevailing wage which explains the 2016 to 2017 decrease from employer wages.

In [31]:
%%sql
SELECT EMPLOYER_STATE, ROUND(AVG(PREVAILING_WAGE_ANNUAL),2) AS AVG_PREVAILING, CASE_SUBMITTED_YEAR
FROM H1B
JOIN EMPLOYER ON H1B.EMPLOYER_KEY = EMPLOYER.KEY
JOIN WAGE ON H1B.WAGE_KEY = WAGE.KEY
JOIN CASES ON H1B.CASE_KEY = CASES.KEY
WHERE EMPLOYER_STATE IN ('CA','PA','TX','NJ','NY') AND CASE_STATUS IN ('CERTIFIED','CERTIFIEDWITHDRAWN')
GROUP BY 1, 3
ORDER BY 1 ASC, 3 ASC;

 * postgresql://student@/week12
28 rows affected.


employer_state,avg_prevailing,case_submitted_year
CA,70907.00,2011
CA,72629.40,2012
CA,85230.20,2013
CA,87778.19,2014
CA,92193.57,2015
CA,97219.97,2016
CA,89017.82,2017
NJ,68674.67,2012
NJ,59350.71,2013
NJ,63467.08,2014


### Conclusions

* Along with the conclusions above, we can conclude the following:
    * Different data elements (willful violator, month, and wage) can be used to see patterns in the case_status denials and approvals.
    * We were able to identify an upward trend in the demand in labor for top companies and industries and identify top positions and industries that these companies were in.
    * We were able to identify differences in wages over employers, locations, and over time. We also were able to perform analyes that may be useful to companies interseted in the wages of their competitors. 

#### Each group member contributed to all parts of the project including the powerpoint.  We worked offline on three occaisions and online together to compolete the project.  Each member specifically contributed the code for one of the four tables dimension tables.  Each member also contributed significantly to the queries for the business questions. For the presentation Miranda will present the introduction, Michael will present business question 1, Maxine will present business question 2, and Zili will present the third business question. 